In [1]:
import csv
import pickle
import numpy

In [2]:
import lmdb
import cv2
import numpy as np

In [3]:
labels = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 
          'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [4]:
mids = ['/m/0cmf2', '/m/0199g', '/m/015p6', '/m/019jd', '/m/04dr76w', '/m/01bjv', '/m/0k4j', 
        '/m/01yrx', '/m/01mzpv', '/m/01xq0k1', '/m/0h8n5zk', '/m/0bt9lr', '/m/03k3r', '/m/04_sv', 
        '/m/01g317', '/m/03fp41', '/m/07bgp', '/m/03m3pdh', '/m/07jdr', '/m/02522']

In [5]:
len(labels), len(mids), 

(20, 20)

In [6]:
label2mid = {}
for i in range(20):
    label2mid[labels[i]] = mids[i]

In [7]:
det_results = {}
pick = False
with open('/dvmm-filer2/projects/AIDA/Ram/USC_IRIS_detections_bbox_coordinates_with_feature_vectors_on_columbia_test_subset_pascal_1.tab', 'r') as fin:
    for line in fin:
        if pick:
            pick = False
        else:
            pick = True
            continue
            
        docid, entev, label, x1, y1, x2, y2, score = line.split()
        
        label = label.split('/')[-1]
        if label not in label2mid:
            continue
        
        val = {
            'label': label2mid[label],
            'score': float(score),
            'bbox': numpy.asarray([x1, y1, x2, y2], dtype='int32'),
        }
        
        if det_results.get(docid) is None:
            det_results[docid] = []
            
        det_results[docid].append(val)

In [8]:
url2id = {}
with open('../../../../data/OpenImages/metadata-v4/image-urls/test-images-with-rotation.csv', 'r') as fin:
    for line in csv.reader(fin):
        if url2id.get(line[2]) != None:
            print('duplicate')
        url2id[line[2]] = line[0]

In [9]:
det_results_2 = {url2id[key]: val for key, val in det_results.items()}

In [10]:
with open('../../temp/test_subset_pascal_1.pkl', 'rb') as fin:
    img_subset_pascal = pickle.load(fin)

In [11]:
lmdb_path = '../../../../data/OpenImages/lmdb/test-600.lmdb'
env = lmdb.open(lmdb_path, map_size=1e11, readonly=True, lock=False)
txn = env.begin(write=False)
cursor = txn.cursor()

In [14]:
image_shape = {}
for i, imgid in enumerate(img_subset_pascal):
    imgbin = txn.get(imgid.encode('utf-8'))
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    h, w, _ = imgbgr.shape
    image_shape[imgid] = (w, h)

/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [15]:
with open('../../temp/pascal_reference_imgsize_1', 'wb') as fout:
    pickle.dump(image_shape, fout)

In [11]:
with open('../../temp/pascal_reference_imgsize_1', 'rb') as fin:
    image_shape = pickle.load(fin)

In [12]:
det_results_3 = {}
for imgid in img_subset_pascal:
    det_results_3[imgid] = []
    if imgid not in det_results_2:
        print(f'results for {imgid} not found')
        continue
    for det in det_results_2[imgid]:
        det_results_3[imgid].append({
            'label': det['label'],
            'score': det['score'],
            'bbox': det['bbox'],
            'bbox_normalized': numpy.asarray([
                det['bbox'][0] / image_shape[imgid][0],
                det['bbox'][1] / image_shape[imgid][1],
                det['bbox'][2] / image_shape[imgid][0],
                det['bbox'][3] / image_shape[imgid][1],
            ]),
            'model': 'pascal',                       
        })

results for 04a6eb882c39a5ce not found
results for a0e244d43b080f3e not found
results for 6f9d7a5776695ab1 not found
results for 9a5e89dd7b708a87 not found
results for 53f2cb95c5152a9b not found
results for 56c26c0e4d8b8829 not found
results for 6c93d3bf2af95172 not found
results for 855808d6566049cc not found
results for 88a73ab97e603b26 not found
results for a66a8a4db9cdf1d5 not found
results for 2f0a4f7fc284420a not found
results for 9e66e8b9b7dda141 not found
results for af81de4df0d72c6b not found
results for 15dcc2cb5c2f72b2 not found
results for a758342da34c2c98 not found
results for 82d61900e7759285 not found
results for 1eb92a30c168a3a5 not found
results for 5279cf374d2308c3 not found
results for ff94d46fcedd8f78 not found
results for b8bf2650c78c81f9 not found
results for a14c67eabcddaa81 not found
results for 7f5fb52baffdcf55 not found
results for 9865400d711fbbfb not found
results for c862df01b724f924 not found
results for 99a0c49a6c22df59 not found
results for c38074e97ca95

In [13]:
det_results_3['8d83dc2e31aa0a6e']

[{'label': '/m/0k4j',
  'score': 0.9746142,
  'bbox': array([  5,  34, 576, 781], dtype=int32),
  'bbox_normalized': array([0.00833333, 0.03777778, 0.96      , 0.86777778]),
  'model': 'pascal'},
 {'label': '/m/0k4j',
  'score': 0.87123984,
  'bbox': array([ 25, 536, 597, 899], dtype=int32),
  'bbox_normalized': array([0.04166667, 0.59555556, 0.995     , 0.99888889]),
  'model': 'pascal'}]

In [14]:
with open('../../results/det_results_ram_reference_1', 'wb') as fout:
    pickle.dump(det_results_3, fout)